In [7]:
import requests
from bs4 import BeautifulSoup

URL = "https://careers.ti.com/en/sites/CX/jobs"
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

jobs = []

# TRY job-list-item__job-info-item
# for job_elem in soup.find_all(class_="job-list-item__job-info-label--posting-date"):
for job_elem in soup.find_all(class_="job-list-item__job-info-item"):
    # title = job_elem.select_one(".job-title").text
    # link = job_elem.select_one("a")["href"]
    # jobs.append({"title": title, "link": link})
    print("HELLO")
    jobs.append(job_elem.text)


In [8]:
print("JOBS arr=")
print(jobs)

JOBS arr=
[]


In [63]:
import asyncio
from playwright.async_api import async_playwright
from datetime import datetime, timedelta
import json
import os

STATE_FILE = "./scraped_jobs.json"

URL = "https://github.com/SimplifyJobs/New-Grad-Positions/tree/dev"
print(URL)

# Check the MAX_ROWS_TO_CHECK top (most recent) rows from the website
MAX_ROWS_TO_CHECK = 10

# All the jobs that the scraper ALREADY has found on past runs of this file ( test.py ).
past_jobs = []
if os.path.exists(STATE_FILE):
    try:
        with open(STATE_FILE, "r") as f:
            past_jobs = json.load(f)
    except Exception as e:
        past_jobs = []

# 
last_past_job = None
if len(past_jobs) > 0:
    # Get the most recent job, out of all the jobs that had already been collected on previous runs of test.py .
    last_past_job = past_jobs[-1]

jobs_to_add = []

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(URL)
        print(await page.title())
        
        await page.goto(URL, wait_until="networkidle")
        await page.wait_for_selector("markdown-accessiblity-table > table > tbody > tr")
        # await page.wait_for_selector("li.job-list-item__job-info-item")
        print("AFTER waiting for markdown-accessiblity-table > table > tbody > tr to appear / be loaded on the page")

        tableRows = page.locator("markdown-accessiblity-table > table > tbody > tr")
        tableRows_num = await tableRows.count()
        print("NUMBER OF ...posting-date tableRows=", tableRows_num)
        tableRows = await tableRows.all()

        numRowsChecked = 0

        currentTime = datetime.now()
        currentDay = datetime.today()
        print("CURRENT DAY=", currentDay)

        jobs_to_add = []
        
        for currentTableRow in tableRows:
            print("START OF THE " + str(numRowsChecked+1) + "TH ITERATION")
            if numRowsChecked > MAX_ROWS_TO_CHECK:
                print("STOPPED SEARCHING FOR JOBS ON THE WEBSITE, BECAUSE I SCRAPED THE LIMIT OF=", numRowsChecked, "JOBS PER RUN OF ME (test.py)")
                break
            print("\n\ncurrentTableRow=", currentTableRow)
            currentTableRowHTML = await currentTableRow.evaluate("element => element.innerHTML")
            print(currentTableRowHTML)

            # Extract all <td> innerHTML into a Python list
            td_values = await currentTableRow.locator("td").evaluate_all(
                "nodes => nodes.map(n => n.innerText)"
            )

            # Check if the 4th column of this row actually has a hyperlink (has a <a> element)
            anchor = currentTableRow.locator("td").nth(3).locator("a")
            url = None
            simplify_url = None
            
            if await anchor.count() > 0:
                # Get the URL of the 1st column of the row, if it exists.
                url = await anchor.first.get_attribute("href")
                if await anchor.count() > 1:
                    simplify_url = await anchor.nth(1).get_attribute("href")                

            print("td_values")
            print(td_values)  # Example output: ['Alice', 'Bob', 'Charlie']

            td_values[3] = {}
            td_values[3]["url"] = url
            td_values[3]["simplify_url"] = simplify_url

            try:
                index = td_values[4].index("d")
                print(f"The first occurrence of 'd' is at index: {index}")
                td_values[4] = td_values[4][0:index]
                try:
                    td_values[4] = int(td_values[4])
                except ValueError:
                    print(f"Error: '{td_values[4]}' cannot be converted to an integer.")
            except ValueError:
                print(f"'d' not found in the string.")

            print("url=")
            print(url)

            print("simplify_url=")
            print(simplify_url)

            # CHECK IF THE CURRENT JOB WE'RE LOOKING AT FROM THE WEBSITE IS ACTUALLY THE MOST-RECENT JOB THAT WE COLLECTED
            # IF YES, THEN WE CAN STOP LOOKING AT JOBS FROM THE WEBSITE OLDER THAN THIS ONE.
            if (last_past_job is not None):
                if (td_values[0] == last_past_job["company"] and td_values[1] == last_past_job["role"] and td_values[2] == last_past_job["location"]):
                    print("STOPPED SEARCHING FOR JOBS ON THE WEBSITE, BECAUSE I SCRAPED A JOB THAT A PREVIOUS CALL OF ME HAD ALREADY FOUND AND WROTE INTO scraped_jobs.json")
                    break

            day_posted = td_values[4]
            if isinstance(td_values[4], int):
                day_posted = currentDay - timedelta(days=td_values[4])
                day_posted = day_posted.strftime("%m-%d-%Y")

            # Add the just-scraped-from-website job to the json file.
            jobs_to_add.append({"company": td_values[0], "role": td_values[1], "location": td_values[2], "links": td_values[3], "day_posted": day_posted})
            
            numRowsChecked += 1
            # END OF THIS ITER OF THE FOR LOOP; MOVE ON TO THE NEXT ROW.
        

       
        await browser.close()

        jobs_to_add = past_jobs + jobs_to_add
        with open(STATE_FILE, "w") as f:
            json.dump(list(jobs_to_add), f, indent=2)

# await main()
if __name__ == "__main__":
    asyncio.run(main())

https://github.com/SimplifyJobs/New-Grad-Positions/tree/dev
GitHub - SimplifyJobs/New-Grad-Positions: A collection of full time roles in SWE, Quant, and PM for new grads.
AFTER waiting for markdown-accessiblity-table > table > tbody > tr to appear / be loaded on the page
NUMBER OF ...posting-date tableRows= 865
CURRENT DAY= 2025-10-02 22:34:51.951402
START OF THE 1TH ITERATION


currentTableRow= <Locator frame=<Frame name= url='https://github.com/SimplifyJobs/New-Grad-Positions/tree/dev'> selector='markdown-accessiblity-table > table > tbody > tr >> nth=0'>

<td><strong><a href="https://simplify.jobs/c/ZipRecruiter?utm_source=GHList&amp;utm_medium=company" rel="nofollow">ZipRecruiter</a></strong></td>
<td>Software Engineer – New Grad - Multiple Teams</td>
<td>Santa Monica, CA</td>
<td><div align="center" dir="auto"><a href="https://job-boards.greenhouse.io/ziprecruiter/jobs/7203130?utm_source=Simplify&amp;ref=Simplify" rel="nofollow"><img src="https://camo.githubusercontent.com/f5ff8ad

In [30]:
import asyncio
from playwright.async_api import async_playwright
from datetime import date

URL = "https://careers.ti.com/en/sites/CX/jobs"
print(URL)

MAX_ITERS = 1

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(URL)
        print(await page.title())

        shadow_page = page.locator("plasmo-csui#jobright-helper-plugin")
        print("shadow_page=", shadow_page)

        frame = page.frame_locator("iframe")
        print("frame=", frame)
        
        await page.goto(URL, wait_until="networkidle")
        await page.wait_for_selector(".app__subpage.app__subpage--custom-page")
        # await page.wait_for_selector("li.job-list-item__job-info-item")
        print("AFTER waiting for li.job-list-item__job-info-item to appear / be loaded on the page")

        linkContainers = page.locator(".app__subpage.app__subpage--custom-page")
        linkContainers = await linkContainers.all()
        for linkContainer in linkContainers:
            print("linkContainer=", linkContainer)
        
        print("FINDING THE PARENT search-job-results CONTAINER")
        linkContainers = page.locator(".search-job-results")
        linkContainers = await linkContainers.all()
        for linkContainer in linkContainers:
            print("linkContainer=", linkContainer)
            
        print("FINDING THE INDIVIDUAL DATE OF A JOB POSTING")
        linkContainers = page.locator("li.job-list-item__job-info-item")
        linkContainers_num = await linkContainers.count()
        print("NUMBER OF job-list-item__job-info-item linkContainers=", linkContainers_num)
        # linkContainers = await linkContainers.all()
        # for linkContainer in linkContainers:
        #     print("\n\nlinkContainer=", linkContainer)
        #     linkContainerHTML = await linkContainer.evaluate("element => element.innerHTML")
        #     print(linkContainerHTML)

        # linkContainers = page.locator("div.job-list-item__job-info-label.job-list-item__job-info-label--posting-date") # Found 25 of these
        # linkContainers = page.locator("div.job-list-item__job-info-value-container > div.job-list-item__job-info-value")
        # Found 50 of these (because it also finds the Job's Location <posting-locations elements)
        
        linkContainers = page.locator("div.job-list-item__job-info-label.job-list-item__job-info-label--posting-date + div div.job-list-item__job-info-value")
        
        linkContainers_num = await linkContainers.count()
        print("NUMBER OF ...posting-date linkContainers=", linkContainers_num)
        linkContainers = await linkContainers.all()
        
        for linkContainer in linkContainers:
            print("\n\nlinkContainer=", linkContainer)
            linkContainerHTML = await linkContainer.evaluate("element => element.innerHTML")
            print(linkContainerHTML)

       
        await browser.close()

await main()

https://careers.ti.com/en/sites/CX/jobs
Search Jobs - Texas Instruments Careers
shadow_page= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='plasmo-csui#jobright-helper-plugin'>
frame= <FrameLocator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='iframe'>
AFTER waiting for li.job-list-item__job-info-item to appear / be loaded on the page
linkContainer= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='.app__subpage.app__subpage--custom-page >> nth=0'>
FINDING THE PARENT search-job-results CONTAINER
linkContainer= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='.search-job-results >> nth=0'>
FINDING THE INDIVIDUAL DATE OF A JOB POSTING
NUMBER OF job-list-item__job-info-item linkContainers= 50
NUMBER OF ...posting-date linkContainers= 25


linkContainer= <Locator frame=<Frame name= url='https://careers.ti.com/en/sites/CX/jobs'> selector='div.job-list-it

In [31]:
# -*- coding: utf-8 -*-
"""
Email to SMS Gateway - config
@author: adam getbags
"""

senderEmail = 'confidencezdata@gmail.com'
gatewayAddress = '2144025928@txt.att.net'
# gatewayAddress = '1112224444@vmobl.com'
appKey = 'lplq bazt ijak ugkw'

In [46]:
# -*- coding: utf-8 -*-
"""
Just send a email, do NOT try to send a message
"""
senderEmail = 'confidencezdata@gmail.com'
gatewayAddress = 'confidencezdata@gmail.com'
appKey = 'lplq bazt ijak ugkw'

import smtplib
from email.message import EmailMessage
# from emailToSMSConfig import senderEmail, gatewayAddress, appKey

msg = EmailMessage()

msg['From'] = senderEmail # 'email@address.com'
msg['To'] = gatewayAddress  # '1112223333@txt.att.net'
msg['Subject'] = 'Job Update'
print("AFTER SETTING THE FIELDS OF THE msg")

msg.set_content("2 Hello! This is an automated email sent using Python.")
print("AFTER calling msg.set_content")

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login(senderEmail, appKey)
    smtp.send_message(msg)

print("Email sent successfully!")

# server = smtplib.SMTP('smtp.gmail.com', 587)
# server.starttls()
# server.login(senderEmail, appKey)
# print("AFTER CALLING server.login with the sender's email address")

# server.send_message(msg)
# print("AFTER CALLING server.send_message with the message msg")
# server.quit()
# print("AFTER CALLING server.quit")

AFTER SETTING THE FIELDS OF THE msg
AFTER calling msg.set_content
Email sent successfully!


In [43]:
import email, smtplib, ssl
# from providers import PROVIDERS
PROVIDERS = {
    "AT&T": {"sms": "txt.att.net", "mms": "mms.att.net", "mms_support": True},
}

# used for MMS
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from os.path import basename

def send_sms_via_email(
    number: str,
    message: str,
    provider: str,
    sender_credentials: tuple,
    subject: str = "sent using etext",
    smtp_server: str = "smtp.gmail.com",
    smtp_port: int = 465,
):
    sender_email, email_password = sender_credentials
    receiver_email = f'{number}@{PROVIDERS.get(provider).get("sms")}'

    # email_message = f"Subject:{subject}\nTo:{receiver_email}\n{message}"
    email_message = f"Subject:{subject}\nTo:{receiver_email}\n{message}"
    print("email_message=", email_message)

    with smtplib.SMTP_SSL(
        smtp_server, smtp_port, context=ssl.create_default_context()
    ) as email:
        email.login(sender_email, email_password)
        email.sendmail(sender_email, receiver_email, email_message)

def send_mms_via_email(
    number: str,
    message: str,
    file_path: str,
    mime_maintype: str,
    mime_subtype: str,
    provider: str,
    sender_credentials: tuple,
    subject: str = "sent using etext",
    smtp_server: str = "smtp.gmail.com",
    smtp_port: int = 465,
):

    sender_email, email_password = sender_credentials
    receiver_email = f'{number}@{PROVIDERS.get(provider).get("sms")}'
    print("receiver_email=", receiver_email)

    email_message = MIMEMultipart()
    # email_message["Subject"] = subject
    email_message["From"] = sender_email
    email_message["To"] = receiver_email

    email_message.attach(MIMEText(message, "plain"))

    # with open(file_path, "rb") as attachment:
    #     part = MIMEBase(mime_maintype, mime_subtype)
    #     part.set_payload(attachment.read())

    #     encoders.encode_base64(part)
    #     part.add_header(
    #         "Content-Disposition",
    #         f"attachment; filename={basename(file_path)}",
    #     )

    #     email_message.attach(part)

    text = email_message.as_string()

    print("BEFORE the with smtplib.SMTP_SSL() as email")
    with smtplib.SMTP_SSL(
        smtp_server, smtp_port, context=ssl.create_default_context()
    ) as email:
        email.login(sender_email, email_password)
        email.sendmail(sender_email, receiver_email, text)
    print("AFTER the with smtplib.SMTP_SSL() as email")
    print("END of send_mms_via_email")


def main():
    number = "2144025928"
    # number = "4698060932"
    message = "hello world!"
    provider = "AT&T"

    sender_credentials = ("confidencezdata@gmail.com", appKey)

    # SMS
    # send_sms_via_email(number, message, provider, sender_credentials)

    # MMS
    # file_path = "/path/to/file/file.png"
    file_path = None

    # mime_maintype = "image"
    # mime_subtype = "png"
    mime_maintype = None
    mime_subtype = None

    # send_mms_via_email(
    #     number,
    #     message,
    #     file_path,
    #     mime_maintype,
    #     mime_subtype,
    #     provider,
    #     sender_credentials,
    # )

    send_sms_via_email(number, message, provider, sender_credentials)


main()

email_message= Subject:sent using etext
To:4698060932@txt.att.net
hello world!
